In [1]:
# !wget "https://www.dropbox.com/scl/fi/acun1rm43ge7ljr5qo6p2/wlasl.zip?rlkey=4o90zt8bhip49m7nows9gcsc8&dl=0"
# !pip install gdown
# !conda install -y gdown
# !gdown --id "1e7nvLl2sdQO9VZDxYbWAZL9UBfUGmONv"

In [2]:
# !mv wlasl.zip* /wlasl.zip
# !unzip -qq /wlasl.zip -d dw-data
# !mv dw-data/data data
# !rm -r dw-data
# !rm wlasl.zip*
# !rm -r sample_data
# !git clone -b feature/mediapipe https://github.com/sceredi/VAR-wlals-recognition.git ./code
# !mv ./code/* ./
# !rm -r code

In [3]:
# !pip install -r requirements.txt
# !pip install mediapipe==0.10.9

In [4]:
import pandas as pd
import numpy as np

from handcrafted.app.dataset.dataset import Dataset 
from wlasl_mediapipe.app.mp.mp_video import MediapipeVideo

2024-04-03 11:12:36.550540: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-03 11:12:36.552805: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 11:12:36.577079: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-03 11:12:36.577131: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-03 11:12:36.577788: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [5]:
def split_data(word_number: int):
  dataset = Dataset('data/WLASL_v0.3.json')
  glosses = pd.read_csv("data/wlasl_class_list.txt", sep="\t", header=None)[1].tolist()
  glosses = glosses[:word_number]
  train_videos = dataset.get_videos(
    lambda video: (video.split == "train") and video.gloss in glosses
  )
  val_videos = dataset.get_videos(
    lambda video: (video.split == "val") and video.gloss in glosses
  )
  test_videos = dataset.get_videos(
    lambda video: (video.split == "test") and video.gloss in glosses
  )
  train_videos = [MediapipeVideo(video, plot=False) for video in train_videos]
  val_videos = [MediapipeVideo(video, plot=False) for video in val_videos]
  test_videos = [MediapipeVideo(video, plot=False) for video in test_videos]
  return train_videos, val_videos, test_videos, glosses

In [6]:
word_number = 6
train_videos, val_videos, test_videos, glosses = split_data(word_number)

/home/simone/uni-lab/VAR/wlasl-recognition/wlasl_mediapipe/app/mp/models/face_model.py:54: RuntimeWarning: invalid value encountered in arccos
  return np.arccos(dot_product / norm)


In [7]:
Y_train = [glosses.index(video.get_base_video().gloss) for video in train_videos]
Y_val = [glosses.index(video.get_base_video().gloss) for video in val_videos]
Y_test = [glosses.index(video.get_base_video().gloss) for video in test_videos]

In [8]:
def concatenate_data(video_list):
    concatenated_data = []
    for video in video_list:
        frames_data = []
        for i in range(len(video.sign_model.left_hand_list)):
            left_hand_data = video.sign_model.lh_embedding[i]
            right_hand_data = video.sign_model.rh_embedding[i]
            pose_data = video.sign_model.pose_embedding[i]
            print(len(pose_data))
            face_data = video.sign_model.face_embedding[i]
            print(len(face_data))
                
            frame_data = np.concatenate((left_hand_data, right_hand_data, pose_data, face_data))
            frames_data.append(frame_data)
        concatenated_data.append(frames_data)
    return concatenated_data


In [9]:
X_train_concatenated = concatenate_data(train_videos)
X_val_concatenated = concatenate_data(val_videos)
X_test_concatenated = concatenate_data(test_videos)


25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
64
25
6

In [10]:
print(X_train_concatenated[0][0].shape)

(971,)


# Model definition
## Libraries useful for ml

In [11]:

from tensorflow import keras
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam

## Preparing the data

In [12]:
# Convert your concatenated data to RaggedTensors
X_train_ragged = tf.ragged.constant(X_train_concatenated, dtype=tf.float32)
X_val_ragged = tf.ragged.constant(X_val_concatenated, dtype=tf.float32)
X_test_ragged = tf.ragged.constant(X_test_concatenated, dtype=tf.float32)

In [13]:
Y_train_one_hot = to_categorical(Y_train, num_classes=word_number)
Y_val_one_hot = to_categorical(Y_val, num_classes=word_number)
Y_test_one_hot = to_categorical(Y_test, num_classes=word_number)

In [14]:
batch_size = 32
X_train_dataset = tf.data.Dataset.from_tensor_slices((X_train_ragged, Y_train_one_hot)).shuffle(buffer_size=X_train_ragged.shape[0]).batch(batch_size)
X_val_dataset = tf.data.Dataset.from_tensor_slices((X_val_ragged, Y_val_one_hot)).shuffle(buffer_size=X_val_ragged.shape[0]).batch(batch_size)
X_test_dataset = tf.data.Dataset.from_tensor_slices((X_test_ragged, Y_test_one_hot)).shuffle(buffer_size=X_test_ragged.shape[0]).batch(batch_size)

In [15]:
print(Y_test_one_hot)

[[0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]


### Defining the model

In [16]:
def build_rnn(input_shape, gru_units_per_layer=[256, 256], output_count=2000,neuron_count_per_hidden_layer=[128,128],activation='relu'):
  model = keras.Sequential()
  model.add(layers.Input(shape=input_shape, ragged=True))
  print(gru_units_per_layer[:-1])
  print(gru_units_per_layer[-1])
  for gru_units in gru_units_per_layer[:-1]:
    model.add(layers.GRU(units = gru_units, return_sequences=True, activation=activation))
  
  model.add(layers.GRU(units = gru_units_per_layer[-1], activation=activation))

  for n in neuron_count_per_hidden_layer:
    model.add(layers.Dense(n,activation=activation))

  model.add(layers.Dense(output_count, activation="softmax"))
  return model

### Model creation

In [17]:
print(len(X_train_ragged[0][0]))

971


In [18]:
input_shape = (None, len(X_train_ragged[0][0]))
model = build_rnn(
    input_shape=input_shape,
    gru_units_per_layer=[256, 256, 256],
    output_count=word_number,
    neuron_count_per_hidden_layer=[128, 64, 32]
)
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

[256, 256]
256
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 256)         943872    
                                                                 
 gru_1 (GRU)                 (None, None, 256)         394752    
                                                                 
 gru_2 (GRU)                 (None, 256)               394752    
                                                                 
 dense (Dense)               (None, 128)               32896     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 6)   

In [19]:
keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## Model fitting

In [20]:
n_epochs = 100
patience = 10

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, restore_best_weights=True)
model.fit(
    X_train_dataset,
    validation_data=X_val_dataset,  
    epochs=n_epochs,
    batch_size=batch_size,
    callbacks=[early_stop],
)

Epoch 1/100
2/2 [==============================] - 3s 558ms/step - loss: 1.7532 - accuracy: 0.2075 - val_loss: 1.7786 - val_accuracy: 0.3077
Epoch 2/100
2/2 [==============================] - 0s 188ms/step - loss: 1.7192 - accuracy: 0.1887 - val_loss: 1.8434 - val_accuracy: 0.2308
Epoch 3/100
2/2 [==============================] - 0s 183ms/step - loss: 1.6153 - accuracy: 0.2830 - val_loss: 2.0241 - val_accuracy: 0.1538
Epoch 4/100
2/2 [==============================] - 0s 181ms/step - loss: 1.5276 - accuracy: 0.3208 - val_loss: 1.9920 - val_accuracy: 0.1538
Epoch 5/100
2/2 [==============================] - 0s 184ms/step - loss: 1.3728 - accuracy: 0.3962 - val_loss: 2.0337 - val_accuracy: 0.2308
Epoch 6/100
2/2 [==============================] - 0s 184ms/step - loss: 1.2748 - accuracy: 0.4151 - val_loss: 2.3396 - val_accuracy: 0.2308
Epoch 7/100
2/2 [==============================] - 0s 184ms/step - loss: 1.2172 - accuracy: 0.4528 - val_loss: 2.0800 - val_accuracy: 0.2308
Epoch 8/100
2

## Model predictions

In [21]:
results = model.evaluate(X_train_dataset)
print("test loss, test acc:", results)

2/2 [==============================] - 0s 59ms/step - loss: 1.6722 - accuracy: 0.2264
test loss, test acc: [1.672176718711853, 0.22641509771347046]


In [22]:
Y_pred = model.predict(X_test_dataset)
accuracy = np.mean(np.argmax(Y_pred, axis=1) == Y_test)
print(f"Accuracy: {accuracy}")

1/1 [==============================] - 0s 325ms/step
Accuracy: 0.3333333333333333


In [23]:
print(np.argmax(Y_pred, axis=1))
print(Y_test)

[3 2 2 3 2 3]
[1, 2, 3, 3, 3, 5]
